In [1]:
from pathlib import Path

import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

In [2]:
DATA_ROOT = Path('..') / 'data'

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])



In [3]:
# Set up some colors for the sensors and the activities.

colors = px.colors.qualitative.D3
values = ['front', 'lateral', 'vertical']
SENSOR_COLORS = {
    value: colors[i] for i, value in enumerate(values)}

colors = px.colors.qualitative.Pastel
values = (sensor_df[['activity', 'activity_label']]
          .drop_duplicates()
          .to_dict(orient='records'))
ACTIVITY_COLORS = {}
for i, value in enumerate(values):
    ACTIVITY_COLORS[value['activity']] = ACTIVITY_COLORS[value['activity_label']] = colors[i]

### Sensor densities by activity

We plot probability densities from the raw data showing all sensors for each activity.
Plotly provides an interactive legend which can be clicked to show or hide the data
for each sensor.

In [6]:
def plot():
    sensors = ['front', 'lateral', 'vertical']
    activity_labels = sorted(sensor_df['activity_label'].unique())
    fig = make_subplots(
        rows=2, 
        cols=2, 
        vertical_spacing=0.1,
        horizontal_spacing=0.1,
        subplot_titles=[label.capitalize() for label in activity_labels])
    
    for index, activity_label in enumerate(activity_labels):
        df = sensor_df[sensor_df['activity_label'] == activity_label]
        for sensor in sensors:
            data = df[sensor]
            density = gaussian_kde(data)
            x = np.linspace(data.min(), data.max(), 250)
            y = density(x)
            fig.add_trace(
                go.Scatter(
                    x=x, 
                    y=y, 
                    line_color=SENSOR_COLORS[sensor],
                    mode='lines', 
                    name=sensor, 
                    showlegend=index == 0,
                    fill='toself',
                    opacity=1.0,
                    legendgroup=sensor),
                col=(index % 2) + 1, row=index // 2 + 1)
    fig.update_layout(
        height=600, 
        width=800,
        margin=dict(l=24, r=24, t=64, b=0),
        legend_title_text='<b>Sensor</b>',
        title='<b>Sensor densities for different activities</b>')
    return fig

app = JupyterDash(__name__)
app.layout = html.Div(children=[
    dcc.Graph(id='plot', figure=plot(), config={'displayModeBar': False})
])

app.run_server(debug=True, mode='inline')

### Activity densities for each sensor

We plot densities again, this time showing each activity for a single sensor.
Again, the legend is interactive and clicking on an activity will
show or hide the density for that activty on all plots.

In [5]:
def plot():
    sensors = ['front', 'lateral', 'vertical']
    activity_labels = sorted(sensor_df['activity_label'].unique())
    fig = make_subplots(
        rows=3, 
        cols=1, 
        vertical_spacing=0.1,
        subplot_titles=[sensor.capitalize() for sensor in sensors])
    

    for activity_label, activity_df in sensor_df.groupby('activity_label'):
        for index, sensor in enumerate(sensors):
            data = activity_df[sensor]
            density = gaussian_kde(data)
            x = np.linspace(data.min(), data.max(), 250)
            y = density(x)
            fig.add_trace(
                go.Scatter(
                    x=x, 
                    y=y, 
                    line_color=ACTIVITY_COLORS[activity_label], 
                    mode='lines', 
                    name=activity_label, 
                    showlegend=index == 0,
                    fill='toself',
                    legendgroup=activity_label),
                row=index + 1, col=1)
    fig.update_layout(
        height=600, 
        width=500,
        margin=dict(l=24, r=24, t=64, b=32),
        legend_title_text='<b>Activity</b>',
        title='<b>Activity densities for different sensors</b>')
    return fig

app = JupyterDash(__name__)
app.layout = html.Div(children=[
    dcc.Graph(id='plot', figure=plot(), config={'displayModeBar': False}),
])

app.run_server(debug=True, mode='inline')